In [5]:
# --- setup ---
%pip install ipywidgets
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interactive, fixed


Note: you may need to restart the kernel to use updated packages.


In [15]:
# -----------------------------------------
# Case A: Dirichlet–Neumann (CCL25, outward increase)
# -----------------------------------------
def phi_ccl25(s, L, D, kdeg, S0, phi0):
    lam = np.sqrt(D / kdeg)
    A = phi0 - S0 / kdeg
    B = -A * np.tanh(L / lam)
    phi = (S0 / kdeg) + A * np.cosh(s / lam) + B * np.sinh(s / lam)
    return phi

# -----------------------------------------
# Case B: Robin–Neumann (CXCL12, outward decrease)
# -----------------------------------------
def phi_cxcl12(s, L, D, kdeg, S0, phi_blood, gamma):
    lam = np.sqrt(D / kdeg)

    # Solve for A using Robin BC at s=0:
    # -D * phi'(0) = gamma * (phi(0) - phi_blood)
    # General solution: phi = S0/k + A*cosh(s/lam) + B*sinh(s/lam)
    # phi'(s) = (A*sinh(s/lam) + B*cosh(s/lam)) / lam
    # Neumann at L -> A*sinh(L/lam) + B*cosh(L/lam) = 0 => B = -A*tanh(L/lam)
    tanhL = np.tanh(L / lam)
    B = lambda A: -A * tanhL
    # At s=0: phi(0) = S0/k + A
    # phi'(0) = B(A)/lam = -A*tanh(L/lam)/lam
    # Robin: -D * (-A*tanh(L/lam)/lam) = gamma*((S0/k + A) - phi_blood)
    # Solve for A:
    num = gamma * (phi_blood - S0 / kdeg)
    denom = (D / lam) * tanhL - gamma
    A = num / denom
    B_val = B(A)
    phi = (S0 / kdeg) + A * np.cosh(s / lam) + B_val * np.sinh(s / lam)
    return phi

# -----------------------------------------
# Thymocyte chemotactic density with saturating sensitivity
# -----------------------------------------
def thymocyte_density(phi, phi0, chi_param, kappa, DC, C0=1.0):
    # Using the derived exponential–log relation:
    term = (phi - phi0) - kappa * np.log((kappa + phi) / (kappa + phi0))
    C = C0 * np.exp((chi_param / DC) * term)
    return C / np.max(C)

# -----------------------------------------
# Combined chemokine fields
# -----------------------------------------
def phi_add(phi25, phi12):
    return phi25 + phi12

def phi_log(phi25, phi12):
    return np.log(1 + phi25 + phi12)

# -----------------------------------------
# Plotting function
# -----------------------------------------
def plot_profiles(L=1.0, D25=1.0, D12=1.0,
                  kdeg25=0.5, kdeg12=0.5,
                  S025=0.50, S012=0.015,
                  phi025=0.35, phi_blood=0.05, gamma=0.5,
                  chi_param=0.8, kappa=0.5, DC=0.05):
    
    s = np.linspace(0, L, 400)

    # Compute chemokine profiles
    phi25 = phi_ccl25(s, L, D25, kdeg25, S025, phi025)          # increasing outward
    phi12 = phi_cxcl12(s, L, D12, kdeg12, S012, phi_blood, gamma)  # decreasing outward

    # Combined fields
    phi_sum = phi_add(phi25, phi12)
    phi_ln = phi_log(phi25, phi12)

    # Thymocyte density
    C_sum = thymocyte_density(phi_sum, phi_sum[0], chi_param, kappa, DC)
    C_ln = thymocyte_density(phi_ln, phi_ln[0], chi_param, kappa, DC)

    # Plot
    fig, ax = plt.subplots(1, 2, figsize=(13, 4))
    ax[0].plot(s, phi25, 'r', lw=2, label='CCL25 (↑ outward)')
    ax[0].plot(s, phi12, 'b--', lw=2, label='CXCL12 (↓ outward)')
    ax[0].plot(s, phi_sum, 'k', lw=1.5, label='Additive')
    ax[0].plot(s, phi_ln, 'gray', lw=1.5, label='Log-sum')
    ax[0].set_xlabel('Position s (normalized)')
    ax[0].set_ylabel('Chemokine concentration ϕ(s)')
    ax[0].legend(); ax[0].grid(alpha=0.3)

    ax[1].plot(s, C_sum, 'g', lw=2, label='C(s) from additive')
    ax[1].plot(s, C_ln, 'orange', lw=2, label='C(s) from log-sum')
    ax[1].set_xlabel('Position s (normalized)')
    ax[1].set_ylabel('Normalized thymocyte density')
    ax[1].legend(); ax[1].grid(alpha=0.3)

    plt.suptitle('CCL25 increases outward | CXCL12 decreases outward')
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

In [ ]:
# ----------------------------------------------------
# Interactive sliders
# ----------------------------------------------------

interactive_plot = interactive(
    plot_profiles,
    # --- Domain and constants ---
    L=fixed(1.0),
    D25=fixed(1.0), D12=fixed(1.0),
    
    # --- Decay constants ---
    kdeg25=(0.1, 1.0, 0.05),
    kdeg12=(0.1, 1.0, 0.05),
    
    # --- Source terms ---
    S025=(0.2, 0.9, 0.02),     # CCL25 cortical production (larger range)
    S012=(0.005, 0.06, 0.002), # CXCL12 CMJ production
    
    # --- Boundary conditions ---
    phi025=(0.1, 0.8, 0.02),   # CCL25 at CMJ (Dirichlet)
    phi_blood=fixed(0.05),     # perivascular CXCL12 sink
    gamma=(0.2, 1.0, 0.05),    # Robin leakage coefficient (CXCL12)
    
    # --- Chemotaxis parameters ---
    chi_param=(0.3, 1.5, 0.05), # effective sensitivity χ
    kappa=(0.1, 1.0, 0.05),     # saturation constant κ
    DC=(0.01, 0.1, 0.005),      # thymocyte motility coefficient D_C
    
    # --- Default physical setup ---
    phi025_init=fixed(0.35),
)





output = interactive_plot.children[-1]
output.layout.height = '500px'
interactive_plot


interactive(children=(FloatSlider(value=0.5, description='kdeg25', max=1.0, min=0.1, step=0.05), FloatSlider(v…